In [2]:
# Larger LSTM Network
# https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/

import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.
//miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
//miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
//miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
//miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: Fu

In [3]:
# load ascii text and covert to lowercase
filename = "AdventuresOfSherlockHolmes.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

In [4]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)


Total Characters:  561209
Total Vocab:  53


In [5]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  561109


In [6]:


# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [ ]:
# fit the model
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/50
561109/561109 [==============================] - 6903s 12ms/step - loss: 2.5467

Epoch 00001: loss improved from inf to 2.54667, saving model to weights-improvement-01-2.5467-bigger.hdf5
Epoch 2/50
561109/561109 [==============================] - 6726s 12ms/step - loss: 2.1825

Epoch 00002: loss improved from 2.54667 to 2.18247, saving model to weights-improvement-02-2.1825-bigger.hdf5
Epoch 3/50
561109/561109 [==============================] - 6764s 12ms/step - loss: 2.0266

Epoch 00003: loss improved from 2.18247 to 2.02663, saving model to weights-improvement-03-2.0266-bigger.hdf5
Epoch 4/50
561109/561109 [==============================] - 6752s 12ms/step - loss: 1.9388

Epoch 00004: loss improved from 2.02663 to 1.93877, saving model to weights-improvement-04-1.9388-bigger.hdf5
Epoch 5/50
561109/561109 [==============================] - 6756s 12ms/step - loss: 1.8747

Epoch 0000

In [7]:
# load the network weights
filename = "weights-improvement-20-1.6027-extendedmodel.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

### Generate Text

##### Example 1

In [8]:
import sys

int_to_char = dict((i, c) for i, c in enumerate(chars))
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# generate characters
for i in range(100):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" nked on to
it--that it is the clew which will guide you in the solution of
some mystery and the puni "
ess of the sertent. i have aeen a seruice to the side of the side of the side of the side of the sid
Done.


##### Example 2

In [10]:
start2 = numpy.random.randint(80, len(dataX)-1)
pattern = dataX[start2]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# generate characters
for i in range(100):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

Seed:
" hat has
happened."

as he spoke he drew the dog-whip swiftly from the dead man's lap,
and throwing t "
p the street. 
"'what is the street."

"i am a care of the sertent."

"i am so she cay in the sime o

##### Example 3

In [11]:
start3 = numpy.random.randint(40, len(dataX)-1)
pattern = dataX[start3]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# generate characters
for i in range(100):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
"  merryweather perched himself upon a crate, with a
very injured expression upon his face, while holm "
es strndd and a serting of the side of the soall past of the side of the soall past of the side of t
Done.


##### Example 4

In [12]:
start4 = numpy.random.randint(50, len(dataX)-1)
pattern = dataX[start4]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# generate characters
for i in range(100):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" about the
foresight and the moral retrogression?"

sherlock holmes laughed. "here is the foresight," "
 said he, "i have aeen a sertice to the side of the side of the side of the side of the side of the 
Done.


##### Example 5

In [13]:
start5 = numpy.random.randint(89, len(dataX)-1)
pattern = dataX[start5]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# generate characters
for i in range(100):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" to a
moral retrogression, which, when taken with the decline of his
fortunes, seems to indicate some "
 searon. i have a conpidte of the sertent of the side of the side of the side of the side of the sid
Done.
